In [1]:
%load_ext autoreload
%autoreload 2
#os.environ["OMP_NUM_THREADS"] = str(os.cpu_count() // 2)
#os.environ["GOMP_CPU_AFFINITY"] = "granularity=core,compact"
from Scripts.model2 import *
from Scripts.loss import *
from Scripts.results_manager import *
from Scripts.plots import *
from Scripts.dataset import *
from Scripts.trainer import *
from Scripts.inference import *
from Scripts.Onnx_Class import *
from Scripts.lr_finder import *
from Scripts.generate_configs import *
from Scripts.excecute import *
from Scripts.upload_summaries import *

c:\Users\alant\AppData\Local\Programs\Python\Python313\Lib\site-packages\openvino\runtime\__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(


In [2]:
config_path = "Configs"

In [ ]:
training_selected_class(config_path, 'grid')
# metrics_to_db()

In [ ]:
training_all_classes(config_path)
metrics_to_db()

In [ ]:
training_run_folder = "Training_Runs"
inference_output_dir = "Inference_Runs"

inference_model(training_run_folder, inference_output_dir)

In [ ]:

config_paths_all = glob.glob(os.path.join(
    "Training_Runs", "**", "*.yaml"), recursive=True)
summary_metrics_paths_all = glob.glob(os.path.join(
    "Training_Runs", "**", "summary_metrics.json"), recursive=True)
best_student_weight_paths_all = glob.glob(os.path.join(
    "Training_Runs", "**", "*best_student.pth"), recursive=True)

config_paths = [
    p for p in config_paths_all if not any(part.startswith('tf_efficient') for part in p.split(os.sep))]
summary_metrics_paths = [
    p for p in summary_metrics_paths_all if not any(part.startswith('tf_efficient') for part in p.split(os.sep))]
best_student_weight_paths = [
    p for p in best_student_weight_paths_all if not any(part.startswith('tf_efficient') for part in p.split(os.sep))]

In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import glob
import os
import time  # Hinzugefügt: Modul für die Zeitmessung
import matplotlib.pyplot as plt

# 1. ONNX-Modell laden
onnx_model_path = r"onnx_models\STFPM_bottle_mobilenetv4_conv_large.onnx"
sess = ort.InferenceSession(onnx_model_path)

image_path = glob.glob(os.path.join(
    r'Images\bottle\test\broken_large', '*.png'))
for path in image_path:
    image = Image.open(path).convert("RGB")

    img_size = 256
    image = image.resize((img_size, img_size))

    input_data = np.array(image, dtype=np.uint8)
    input_data = np.expand_dims(input_data, axis=0) 

    input_name = sess.get_inputs()[0].name

    start_time = time.perf_counter() 

    outputs = sess.run(None, {input_name: input_data})

    end_time = time.perf_counter()  
    inference_time = end_time - start_time  
    
    anomaly_map = outputs[0]
    anomaly_score = outputs[1]

    print(f"Anomalie-Score für das Bild: {anomaly_score[0]}")
    print(f"Inferenzzeit: {inference_time:.4f} Sekunden")

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # 2. Originalbild anzeigen
    axes[0].imshow(image)
    axes[0].set_title("Originalbild")
    axes[0].axis('off')

    # 3. Heatmap über das Originalbild legen
    axes[1].imshow(image)
    heatmap = axes[1].imshow(np.squeeze(anomaly_map), cmap='jet', alpha=0.5)
    axes[1].set_title("Anomalie-Heatmap")
    axes[1].axis('off')

    # 4. Farbbalken für die Heatmap hinzufügen
    fig.colorbar(heatmap, ax=axes[1], fraction=0.046, pad=0.04)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    # 5. Plot anzeigen oder speichern
    plt.suptitle("Vergleich: Originalbild vs. Anomalie-Heatmap")
    plt.show()

In [4]:
config = load_config(r'Configs\resnet18\grid\STFPM_Config_resnet18_grid.yaml')
model = STFPM(
    architecture=config['model']['architecture'],
    layers=config['model']['layers']
)
print(model.student_model)

FeatureExtractor(
  (model): ResNet(
    (conv1): Identity()
    (bn1): Identity()
    (relu): Identity()
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.

In [6]:
import torchvision.models.quantization

model = torchvision.models.quantization.resnet18(
    weights="DEFAULT", quantize=False)
print(model)

QuantizableResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): QuantizableBasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (add_relu): FloatFunctional(
        (activation_post_process): Identity()
      )
    )
    (1): QuantizableBasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, e